# Creating the containers for .NET notebooks for exploring dbatools

To run these notebooks, you will first have to have set up Jupyter Notebooks following the information here https://sqldbawithabeard.com/2020/02/07/new-net-notebooks-are-here-powershell-7-notebooks-are-here/

![dbatools](.\images\dbatools.png )

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

# Setting up the containers for the rest of the dbachecks notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers. 

This notebook will enable you to do that.

You will need to have Docker installed. The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart.  

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png )

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

**You can also navigate through the Notebook using the arrow keys and press SHIFT + ENTER to run a cell**

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if(Test-Path $FolderPath){
    Write-Output "Path $FolderPath exists already"
}else {
 $null =    New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if(Test-Path "$FolderPath\SQL1"){
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse| Remove-Item -Recurse -Force
}else {
$null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if(Test-Path "$FolderPath\SQL1\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if(Test-Path "$FolderPath\SQL2"){
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse| Remove-Item -Recurse -Force
}else {
 $null =    New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if(Test-Path "$FolderPath\SQL2\dummyfile.txt"){
    Write-Output "dummyfile.txt exists already"
}else {
$null =  New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"
try {
    $dockercompose = (Get-Content .\dockercompose.yml -ErrorAction Stop) -replace '__ReplaceME__' , $FolderPath
    # $dockercompose
    $dockercompose | Set-Content .\docker-compose.yml
}
catch {
    $NotebookDir = Read-Host "Please enter path to the notebook. Azure Data Studio doesnt know where it is"
    $dockercompose = (Get-Content $NotebookDir\\dockercompose.yml -ErrorAction Stop) -replace '__ReplaceME__' , $FolderPath
    # $dockercompose
    $dockercompose | Set-Content $NotebookDir\\docker-compose.yml
    Set-Location $NotebookDir
}

Write-Output "Finished"

Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo


Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL1


Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL1\dummyfile.txt


Creating Directory C:\Users\mrrob\Documents\dbatoolsdemo\SQL2


Creating File C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\dummyfile.txt


Creating a credential file for the containers - Please don't do this in production


Credential file created


Setting the docker-compose files values


Finished


## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbatoolsbeardsql01 [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbatoolsbeardsql01) and sqldbawithabeard\dbatoolsbeardsql02 [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbatoolsbeardsql02)

The first time it is going to pull the images from the Docker Hub. If you wish to do this first, you can run the cell below



In [8]:
docker pull sqldbawithabeard/dbatoolsbeardsql01:v1
docker pull sqldbawithabeard/dbatoolsbeardsql02:v1

v1: Pulling from sqldbawithabeard/dbatoolsbeardsql01


Digest: sha256:33feca7141acba3ba61dd1d44a823fd3689487533d539cbbbd9be755908b78af


Status: Image is up to date for sqldbawithabeard/dbatoolsbeardsql01:v1


docker.io/sqldbawithabeard/dbatoolsbeardsql01:v1


v1: Pulling from sqldbawithabeard/dbatoolsbeardsql02


Digest: sha256:b3a6f73910245bcb5857b03246e032188fc24bc44407ace89e4a97355247ad3d


Status: Image is up to date for sqldbawithabeard/dbatoolsbeardsql02:v1


docker.io/sqldbawithabeard/dbatoolsbeardsql02:v1


Now we can start the containers with the code below

In [2]:
docker-compose up -d

Creating network "dbatools_BeardsMagicNetwork" with the default driver


Creating beardsql01 ... 


Creating beardsql02 ... 


Creating beardsql02 ... done


Creating beardsql01 ... done


All being well, you wil have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way


Now we can start exploring with dbatools :-)

If you have not installed dbatools, it can be got from the PowerShell Gallery using `Install-Module dbatools` the code below will check for the module and either install it in your user profile or update it and Import it

In [2]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

Installing dbatools in your user profile


                                                                                                    
 Installing package 'dbatools'                                                                      
                                                                                                    
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Downloaded 0.00 MB out of 24.91 MB.                                                             
    [                                                                                        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Downloaded 15.00 MB out of 24.91 MB.                                                            
    [oooooooooooooooooooooooooooooooooooo                                                    ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Unzipping                                                                                       
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo                           ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Copying unzipped package to 'C:\Users\mrrob\AppData\Local\Temp\1865382987\dbatools.1.0.113'     
    [oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo        ]      
                                                                                                    

                                                                                                    
 Installing package 'dbatools'                                                                      
    Process Package Manifest                                                                        
    [ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo     ]      
                                                                                                    

Import-Module: 
Line |
   8 |  Import-Module dbatools
     |  ~~~~~~~~~~~~~~~~~~~~~~
     | Could not find a part of the path 'C:\Users\mrrob\OneDrive\Documents\PowerShell\Modules\dbatools\1.0.108\xml\dbatools.Format.ps1xml'.


## Check connection
Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

Import-Clixml: 


Line |


   3 |  $SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred


     |                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


     | Could not find a part of the path 'C:\Users\mrrob\Documents\dbatoolsdemo\sqladmin.cred'.


 Creating connection to the containers


If there are warnings above - Check the containers are running - Look at the status column

In [5]:
docker ps -a

CONTAINER ID        IMAGE                              COMMAND                  CREATED             STATUS                     PORTS                                                                                    NAMES


b0d18d09e3de        sqldbawithabeard/dbachecksdemo     "/opt/mssql/bin/perm…"   54 seconds ago      Up 48 seconds              0.0.0.0:5022->5022/tcp, :::5022->5022/tcp, 0.0.0.0:15592->1433/tcp, :::15592->1433/tcp   dbatools_SQL2019_1


92fb189db4b9        sqldbawithabeard/dbachecksdemo     "/opt/mssql/bin/perm…"   54 seconds ago      Up 48 seconds              0.0.0.0:5023->5023/tcp, :::5023->5023/tcp, 0.0.0.0:15593->1433/tcp, :::15593->1433/tcp   dbatools_SQL2019-1_1


b746ed44faf2        sqldbawithabeard/pwshdbatools:v1   "pwsh"                   12 days ago         Exited (0) 12 days ago                                                                                              vibrant_feistel


5a29c6514f4b        sqldbawithabeard/pwshdbatools:v1   "pwsh"                   12 days ago         Exited (0) 12 days ago                                                                                              frosty_carson


a54110819a3f        sqldbawithabeard/pwshdbatools:v1   "pwsh"                   12 days ago         Exited (0) 12 days ago                                                                                              gifted_elion


153c30cb1841        mcr.microsoft.com/powershell       "pwsh"                   12 days ago         Exited (143) 12 days ago                                                                                            pensive_zhukovsky


ad3e8b0db7c5        jekyll/jekyll:3.8.6                "/usr/jekyll/bin/ent…"   3 months ago        Exited (0) 3 months ago                                                                                             musing_hellman


If there are warnings above and the containers are running - check the logs

In [6]:
docker logs dbatools_SQL2019_1

SQL Server 2019 will run as non-root by default.


This container is running as user mssql.


Your master database file is owned by mssql.


To learn more visit https://go.microsoft.com/fwlink/?linkid=2099216.


2021-08-06 06:23:27.63 Server      Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 


...

2021-08-06 06:23:31.00 spid51      Using 'xpsqlbot.dll' version '2019.150.2000' to execute extended stored procedure 'xp_qv'. This is an informational message only; no user action is required.


2021-08-06 06:23:31.04 spid51      Attempting to load library 'xpstar.dll' into memory. This is an informational message only. No user action is required.


2021-08-06 06:23:31.07 spid51      Using 'xpstar.dll' version '2019.150.2000' to execute extended stored procedure 'xp_sqlagent_notify'. This is an informational message only; no user action is required.


2021-08-06 06:24:08.31 spid52      Attempting to load library 'xplog70.dll' into memory. This is an informational message only. No user action is required.


2021-08-06 06:24:08.34 spid52      Using 'xplog70.dll' version '2019.150.2000' to execute extended stored procedure 'xp_msver'. This is an informational message only; no user action is required.


If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbatools_SQL2019_1

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up